In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import utm
import geojsoncontour
import plotly.graph_objects as go
import laspy
import rasterio
import pyproj
import pygmt
import sys
from affine import Affine
from rasterio.transform import from_origin
from tqdm import tqdm
from matplotlib.patches import Circle
from sklearn.cluster import DBSCAN
from pyproj import Transformer, CRS
from rasterio.enums import Resampling

Credits to https://www.generic-mapping-tools.org/egu22pygmt/lidar_to_surface.html for the tutorial on pygmt and LiDAR data

# Constants

In [2]:
# Define the bounding box coordinates in WGS-84
LAT_MIN, LAT_MAX = 50.85, 51.001
LON_MIN, LON_MAX = 6.85, 7.05

# Minimum Z to consider
Z_MIN = 50

# Subsampling factor for points cloud
SSFACTOR = 4

# Subbox size in meters
box_size = 2000

# Points classification to consider
# https://www.bezreg-koeln.nrw.de/brk_internet/geobasis/hoehenmodelle/nutzerinformationen.pdf
lastReturnNichtBoden = 20
brueckenpunkte = 17
class_ok = [brueckenpunkte, lastReturnNichtBoden]

# Define the functions we need

In [3]:
# Define conversion functions

def utm_to_latlon(x, y):
    # Convert lat/lon to UTM coordinates
    lat, lon = utm.to_latlon(x, y, 32, 'U')

    return lat, lon

def latlon_to_utm(lat, lon):
    # Convert lat/lon to UTM coordinates
    utm_x, utm_y, _, _ = utm.from_latlon(lat, lon)

    return utm_x, utm_y

In [5]:
# Define functions to find and load LiDAR data files

def find_files(bbox_min_x, bbox_max_x, bbox_min_y, bbox_max_y):
       
    bbox_min_x = bbox_min_x
    bbox_min_y = bbox_min_y

    bbox_max_x = bbox_max_x
    bbox_max_y = bbox_max_y

    # Determine the necessary .laz files based on the easting and northing coordinates
    min_easting = int(bbox_min_x) // 1000
    min_northing = int(bbox_min_y) // 1000
    max_easting = int(bbox_max_x) // 1000
    max_northing = int(bbox_max_y) // 1000

    laz_files = []

    for easting in range(min_easting, max_easting + 1):
        for northing in range(min_northing, max_northing + 1):
            #filename = f"./lidar_data/3dm_32_{easting:03d}_{northing:04d}_1_nw.laz"
            filename = f"/Volumes/SSD_portable/lidar_data/Cologne_extended/3dm_kacheln/3dm_32_{easting:03d}_{northing:04d}_1_nw.laz"
            laz_files.append(filename)

    return laz_files

def load_files(laz_files):

    # Initialize empty arrays for point coordinates and elevations
    x_all = np.array([])
    y_all = np.array([])
    z_all = np.array([])

    for file in laz_files:
        las = laspy.read(file)
        #print('processing %s'%(file))
        x = las.x[::SSFACTOR]
        y = las.y[::SSFACTOR]
        z = las.z[::SSFACTOR]
        class_val = las.classification[::SSFACTOR]

        mask = (np.isin(class_val, class_ok))&(z>=Z_MIN)

        # Stack point coordinates and elevations
        x_all = np.hstack((x_all, x[mask]))
        y_all = np.hstack((y_all, y[mask]))
        z_all = np.hstack((z_all, z[mask]))
    
    return x_all, y_all, z_all

In [6]:
# Define functions to load elevation data from the DEM

def transform_coords(lat, lon, src_crs, dest_crs):
    """Transform coordinates from the source CRS to the destination CRS."""
    transformer = pyproj.Transformer.from_crs(src_crs, dest_crs, always_xy=True)
    return transformer.transform(lon, lat)

def latlon_to_rowcol(lat, lon, transform):
    """Convert latitude and longitude to row and column indices."""
    col, row = ~transform * (lon, lat)
    return int(row), int(col)

def get_elevation(dem_file, lat, lon):
    """Get elevation for a given latitude and longitude from a DEM file."""
    with rasterio.open(dem_file) as dataset:
        # Get the affine transform for the dataset
        transform = dataset.transform

        # Define CRS for WGS84 and ETRS89
        wgs84_crs = 'EPSG:4326'
        etrs89_crs = dataset.crs

        # Transform the coordinates from WGS84 to ETRS89
        etrs89_lon, etrs89_lat = transform_coords(lat, lon, wgs84_crs, etrs89_crs)

        # Convert lat, lon to row, col
        row, col = latlon_to_rowcol(etrs89_lat, etrs89_lon, transform)

        # Read the elevation value
        elevation = dataset.read(1)[row, col]
        return elevation

In [22]:
# Function to calculate the Haversine distance (in meters) between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# Function to find the closest point within a threshold distance
def find_closest_within_threshold(AIP_point, validation_data, threshold=10):
    distances = haversine(AIP_point['geoLat'], AIP_point['geoLong'], validation_data['lat'], validation_data['lon'])
    min_distance = np.min(distances)
    if min_distance <= threshold:
        closest_index = np.argmin(distances)
        return closest_index, min_distance
    return None, None

# Load LiDAR data in a dataframe - Do not execute if the dataframe pickle is already available

In [11]:
bbox_min_x, bbox_min_y = latlon_to_utm(LAT_MIN, LON_MIN)
bbox_max_x, bbox_max_y = latlon_to_utm(LAT_MAX, LON_MAX)

x_edges = []
y_edges = []

x_edges = np.arange(bbox_min_x, bbox_max_x,  box_size)
x_edges = np.append(x_edges, bbox_max_x)

y_edges = np.arange(bbox_min_y, bbox_max_y,  box_size)
y_edges = np.append(y_edges, bbox_max_y)

x_len = len(x_edges)
y_len = len(y_edges)

x_all = np.array([])
y_all = np.array([])
z_all = np.array([])

print('x_len: %s, y_len: %s, number of boxes: %s'%(str(x_len), str(y_len), str((x_len-1)*(y_len-1))))

cols, rows = (x_len-1, y_len-1)

# Initialise 2D lists (which will contain arrays and not scalar, so 2D np array does not work here)
x_results = [[0 for i in range(cols)] for j in range(rows)]
y_results = [[0 for i in range(cols)] for j in range(rows)]
z_results = [[0 for i in range(cols)] for j in range(rows)]

# iterate on subboxes
for i in tqdm(range(x_len - 1)): # subboxes along x axis (longitude)
    for j in range(y_len - 1): # subboxes along y axis (latitude)

        laz_files = find_files(x_edges[i], x_edges[i+1], y_edges[j], y_edges[j+1])

        x_all_temp, y_all_temp, z_all_temp = load_files(laz_files)

        x_all = np.concatenate((x_all, x_all_temp))
        y_all = np.concatenate((y_all, y_all_temp))
        z_all = np.concatenate((z_all, z_all_temp))

x_len: 9, y_len: 10, number of boxes: 72


100%|██████████| 8/8 [09:30<00:00, 71.25s/it]


In [14]:
# Create our dataframe

df = pd.DataFrame(
    data={
        "x": x_all,
        "y": y_all,
        "z": z_all
    }
)

size_df = sys.getsizeof(df)
print(f"Size of the DataFrame: {np.ceil(size_df / (1024*1024))} MB")

# Save our dataframe for future use (optional, heavy file)

# df.to_pickle("./lidar_pkl/df_Cologne_extended.pkl")

Size of the DataFrame: 6189.0 MB


# Load the LiDAR dataframe (optional)

In [6]:
df = pd.read_pickle("/Volumes/SSD_portable/lidar_pkl/df_Cologne_extended_SS2.pkl")

# Preprocess LiDAR data using pygmt's blockmedian function

In [7]:
# Get bounding box region

region = pygmt.info(data=df[["x", "y"]], spacing=1)  # West, East, South, North

print(f"Data points covers region: {region}")

Data points covers region: [ 348000.  364000. 5635000. 5652000.]


In [8]:
df_trimmed = pygmt.blockmedian(
    data=df[["x", "y", "z"]],
    T=0.99,  # 99th quantile, i.e. the highest point
    spacing="1+e",
    region=region,
)

size_df_trimmed = sys.getsizeof(df_trimmed)
print(f"Size of the DataFrame: {np.ceil(size_df_trimmed / (1024*1024))} MB")

del(df)

Size of the DataFrame: 1780.0 MB


In [9]:
# Save the trimmed dataframe if necessary

df_trimmed.to_pickle("./lidar_data/df_trimmed_Cologne_extended_SS2.pkl")

# Load the LiDAR trimmed dataframe (optional)

In [7]:
df_trimmed = pd.read_pickle("./lidar_pkl/df_trimmed_Cologne_extended_SS2.pkl")

# Identify the obstacles user DBSCAN clusters

In [8]:
# Indicative time: 1'50"

# Identify points that are above 120m above geoid (for Cologne this means about 70 m above ground).
high_points = df_trimmed[df_trimmed['z'] > 120]

# Assuming that points within 100m of each other belong to the same obstacle
clustering = DBSCAN(eps=50, min_samples=2).fit(high_points[['x', 'y', 'z']])

# Add the cluster labels to the high_points DataFrame
high_points['cluster'] = clustering.labels_

# Filter out noise points (DBSCAN labels noise as -1)
obstacles = high_points[high_points['cluster'] != -1]

# Find the highest point in each obstacle cluster
highest_points = obstacles.loc[obstacles.groupby('cluster')['z'].idxmax()]

# The resulting DataFrame 'highest_points' contains the coordinates of the highest point of each obstacle
highest_points.reset_index(drop=True, inplace=True)

# Apply the conversion function to the DataFrame to create new columns 'lat' and 'lon'
highest_points['lat'], highest_points['lon'] = zip(*highest_points.apply(lambda row: utm_to_latlon(row['x'], row['y']), axis=1))

# Display the resulting DataFrame
pd.set_option('display.max_rows', 200)
highest_points

/var/folders/v_/0jkxx12x7r9_3szb050fks9m0000gn/T/ipykernel_97011/748855324.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_points['cluster'] = clustering.labels_


,x,y,z,cluster,lat,lon
0,355697.36,5651889.23,194.01,0,51.000461,6.943404
1,358812.46,5649010.91,120.50,1,50.975366,6.988890
2,358839.30,5648870.44,121.02,2,50.974111,6.989326
3,358288.03,5647378.10,182.56,3,50.960564,6.982061
4,352499.36,5647160.40,125.89,4,50.957155,6.899778
5,357269.30,5646497.48,121.41,5,50.952398,6.967909
6,356710.33,5646385.06,135.28,6,50.951249,6.960001
7,354861.48,5646335.07,150.30,7,50.950337,6.933717
8,355512.21,5646065.47,215.82,8,50.948078,6.943082
9,354726.28,5645982.39,312.06,9,50.947134,6.931934


In [9]:
highest_points_backup = highest_points.copy()

In [158]:
# Load the backup
highest_points = highest_points_backup.copy()

## Conversions

### DHHN2016

In [10]:
# Transform heights from DHHN2016 (EPSG:7837) to WGS84 ellipsoid (EPSG:4326)
# Path the GeoTIFF file containing the GCG2016 quasigeoid data (source: https://gdz.bkg.bund.de/index.php/default/quasigeoid-der-bundesrepublik-deutschland-quasigeoid.html)

GCG2016_geoid_file = 'GCG2016_data/GCG2016_we.tif'

# Get geoid height from the GeoTIFF file
def get_geoid_height(lat, lon, geoid_file):
    with rasterio.open(geoid_file) as dataset:
        # Coordinate transformation from WGS-84 to ETRS89 (check EPSG code)
        transform = pyproj.Transformer.from_crs(pyproj.CRS('EPSG:4326'), dataset.crs, always_xy=True)
        x, y = transform.transform(lon, lat)
        
        # Read the geoid height at the given coordinates
        row, col = dataset.index(x, y)
        geoid_height = dataset.read(1, window=rasterio.windows.Window(col, row, 1, 1), resampling=Resampling.nearest)
        return geoid_height[0, 0]

# Apply the geoid correction to the DataFrame (the geoid undulation must be added)
highest_points['z_wgs84'] = highest_points.apply(lambda row: row['z'] + get_geoid_height(row['lat'], row['lon'], GCG2016_geoid_file), axis=1)


In [11]:
# Transform heights from WGS84 (ellipsoid) to WGS84 (EGM96 geoid) (EPSG:5773)
# Geoid grid file source: https://github.com/OSGeo/proj-datumgrid
egm96_geoid_file = './EGM96_data/egm96_15.gtx'

# Create a pyproj Transformer to transform from WGS-84 ellipsoidal heights to EGM96 geoid heights
transformer = pyproj.Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +proj=unitconvert +xy_in=deg +z_in=m +xy_out=rad +z_out=m "
    f"+step +proj=vgridshift +grids={egm96_geoid_file} +multiplier=-1 "
    f"+step +proj=unitconvert +xy_in=rad +z_in=m +xy_out=deg +z_out=m"
)

def apply_egm96_transformation(row):
    _, _, z_egm96 = transformer.transform(row['lon'], row['lat'], row['z_wgs84'])
    return z_egm96

# Apply the EGM96 transformation to the DataFrame
highest_points['z_egm96'] = highest_points.apply(apply_egm96_transformation, axis=1)

# test: transformer.transform(7, 5, 0) # https://geographiclib.sourceforge.io/cgi-bin/GeoidEval

# Alternative rasterio method
'''src = rasterio.open(egm96_geoid_file)
lat = 52
lon = 7
geoid_height = next(src.sample([(lon, lat)]))[0]'''

'src = rasterio.open(egm96_geoid_file)\nlat = 52\nlon = 7\ngeoid_height = next(src.sample([(lon, lat)]))[0]'

### EGG08

In [12]:
def load_egg08_grid(file_path):
    """Load EGG08 grid data from a file."""
    grid_data = np.loadtxt(file_path, skiprows=16) # skip the 16 lines corresponding to the header
    return grid_data.reshape((360, 480))  # Reshape based on nrows and ncols

def interpolate_EGG08_height(grid, lat, lon):
    """Interpolate geoid height from the EGG08 grid for a given latitude and longitude."""
    lat_min, lat_max, lon_min, lon_max = 25.0, 85.0, -50.0, 70.0
    delta_lat, delta_lon = 0.1667, 0.2500
    nrows, ncols = 360, 480

    if lat < lat_min or lat > lat_max or lon < lon_min or lon > lon_max:
        return None  # Coordinates are outside the grid bounds

    # Calculate the row and column indices for interpolation
    row = int((lat - lat_min) / delta_lat)
    col = int((lon - lon_min) / delta_lon)

    # Perform the interpolation
    return grid[row, col]

# Load the grid data
egg08_grid = load_egg08_grid('EGG08_data/EGG2008_20170702.isg')

# Add DEM data to our LiDAR obstacles shortlist

In [13]:
dem_file = './DEM_data/Cologne_EUDEM_v11.tif'

# Add DEM elevations to the LiDAR obstacles data and calculate obstacles heights
for index, row in highest_points.iterrows():
    highest_points.at[index, 'dem_gnd_elev'] = get_elevation(dem_file, row['lat'], row['lon'])
for index, row in highest_points.iterrows():
    highest_points.at[index, 'lidar_obs_hgt'] = row['z'] - row['dem_gnd_elev']

In [14]:
# Apply the geoid correction to the DataFrame (the geoid undulation must be added)
highest_points['dem_gnd_elev_wgs84'] = highest_points.apply(lambda row: row['dem_gnd_elev'] + interpolate_EGG08_height(egg08_grid, row['lat'], row['lon']), axis=1)

In [15]:
def apply_egm96_transformation(row):
    _, _, z_egm96 = transformer.transform(row['lon'], row['lat'], row['dem_gnd_elev_wgs84'])
    return z_egm96

# Apply the EGM96 transformation to the DataFrame
highest_points['dem_gnd_elev_egm96'] = highest_points.apply(apply_egm96_transformation, axis=1)

In [16]:
for index, row in highest_points.iterrows():
    highest_points.at[index, 'lidar_obs_hgt'] = row['z_egm96'] - row['dem_gnd_elev_egm96']

In [17]:
highest_points.reset_index(inplace=False)
lidar_validation_data = highest_points[highest_points['lidar_obs_hgt']>70].copy()

# TODO: improvement needed as lidar_obs_hgt is not reliable (due to the poor DEM?)
# Large height difference for Ringturm and smoke stack 163-40, for example

In [18]:
lidar_validation_data.reset_index(inplace=True)
lidar_validation_data.sort_values(by=['lidar_obs_hgt'])

,index,x,y,z,cluster,lat,lon,z_wgs84,z_egm96,dem_gnd_elev,lidar_obs_hgt,dem_gnd_elev_wgs84,dem_gnd_elev_egm96
15,15,356142.25,5645207.48,131.50,15,50.940525,6.952384,177.986000,131.080109,61.790272,70.784728,107.201272,60.295380
39,104,354563.23,5636029.49,122.35,104,50.857648,6.933579,168.921098,121.895430,52.672382,70.837716,98.083382,51.057714
34,69,348011.35,5636612.46,206.76,69,50.861202,6.840315,253.219099,206.291803,136.952194,70.855905,182.363194,135.435898
14,14,356122.33,5645263.32,132.19,14,50.941022,6.952078,178.676000,131.771299,61.924259,71.340741,107.335259,60.430558
5,5,357269.30,5646497.48,121.41,5,50.952398,6.967909,167.904099,120.999171,50.063187,72.429912,95.474187,48.569259
29,39,356483.31,5642672.99,123.69,39,50.917833,6.958233,170.191999,123.241662,51.908123,72.872876,97.319123,50.368786
17,18,350416.35,5645009.33,141.85,18,50.937287,6.871024,188.209001,141.393409,69.182129,73.615872,114.593129,67.777537
4,4,352499.36,5647160.40,125.89,4,50.957155,6.899778,172.263699,125.449985,52.901062,73.951637,98.312062,51.498347
1,1,358812.46,5649010.91,120.50,1,50.975366,6.988890,167.023602,120.129465,47.222523,74.390079,92.633523,45.739386
36,98,357971.49,5636330.17,129.15,98,50.861198,6.981856,175.767199,128.699847,55.731514,74.624685,101.142514,54.075162


# Load AIP data and filter it

In [19]:
AIP_df = pd.read_excel("./AIP_data/ED_Obstacles_Area_1_2023-11-02_2023-11-02_snapshot.xlsx")

AIP_df = AIP_df[['geoLat', 'geoLong', 'txtName', 'Type of Obstacle', 'ValElev (m)', 'valHgt (m)', 'Location (inofficial)', 'Description (inofficial)']]

AIP_df = AIP_df[(AIP_df['geoLat'] >= LAT_MIN) & 
                     (AIP_df['geoLat'] <= LAT_MAX) &
                     (AIP_df['geoLong'] >= LON_MIN) & 
                     (AIP_df['geoLong'] <= LON_MAX)]

In [20]:
AIP_df

,geoLat,geoLong,txtName,Type of Obstacle,ValElev (m),valHgt (m),Location (inofficial),Description (inofficial)
12025,50.941520,6.957296,NORDRHEIN-WESTFALEN 92-10,SPIRE,212.51,157.41,Köln,
12029,51.000449,6.943347,NORDRHEIN-WESTFALEN 98-10,STACK,193.31,150.01,Koeln-Weidenpesch,
12105,50.860811,6.980933,NORDRHEIN-WESTFALEN 163-20,INDUSTRIAL_SYSTEM,225.31,175.01,Godorf,KAMIN KONVERSIONSANLAGE
12106,50.854561,6.980056,NORDRHEIN-WESTFALEN 163-30,INDUSTRIAL_SYSTEM,160.31,110.01,Godorf,KAMIN KRAFTWERK
12107,50.857103,6.983283,NORDRHEIN-WESTFALEN 163-40,INDUSTRIAL_SYSTEM,160.31,110.01,Godorf,KAMIN XYLOL-ANLAGE
12110,50.921800,6.932576,NORDRHEIN-WESTFALEN 167-10,BUILDING,184.71,133.01,Köln,
12138,50.902914,6.960628,NORDRHEIN-WESTFALEN 191-10,ANTENNA,190.76,138.01,Koeln-Raderberg,
12148,50.946944,6.931944,NORDRHEIN-WESTFALEN 204-10,TOWER,316.59,268.09,"Köln (Köln 8, Colonius), Gemark. Ehrenfeld, In...",
12205,50.921617,7.004475,NORDRHEIN-WESTFALEN 263-10,TOWER,160.01,115.01,Koeln-Gremberg,
12213,50.947908,6.942575,NORDRHEIN-WESTFALEN 271-10,ANTENNA,215.16,165.01,"Koeln, Fl.37, Flst.363,365",


In [52]:
validation_data = lidar_validation_data.copy()
AIP_validated_df = pd.DataFrame()
index_used = []

# Process AIP_df
for idx, row in AIP_df.iterrows():
    closest_index, min_distance = find_closest_within_threshold(row, validation_data, 50)
    #print("closted index to %s is %s"%(idx, closest_index))
    #if closest_index is not None: print(validation_data.loc[closest_index, ['lat', 'lon', 'z_egm96', 'dem_gnd_elev_egm96', 'lidar_obs_hgt', 'cluster']])
    new_row = row.to_dict()
    if closest_index is not None:
        new_row.update(validation_data.loc[closest_index, ['lat', 'lon', 'z_egm96', 'dem_gnd_elev_egm96', 'lidar_obs_hgt', 'cluster']].to_dict())
        index_used.append(closest_index)
    else:
        new_row.update({'lat': np.nan, 'lon': np.nan, 'z_egm96': np.nan, 'dem_gnd_elev_egm96': np.nan})
    new_row['delta_Pos'] = min_distance if min_distance is not None else np.nan
    new_row['delta_Hgt'] = new_row['lidar_obs_hgt'] - new_row['valHgt (m)'] if 'lidar_obs_hgt' in new_row else np.nan
    new_row['delta_Elev'] = new_row['z_egm96'] - new_row['ValElev (m)'] if 'z_egm96' in new_row else np.nan
    new_row_df = pd.DataFrame([new_row])
    AIP_validated_df = pd.concat([AIP_validated_df, new_row_df], ignore_index=True)

validation_data.drop(index_used, inplace=True)

# Completing the addition of unused entries from highest_points to AIP_validated_df
for idx, row in validation_data.iterrows():
    new_row = {col: np.nan for col in AIP_df.columns}
    new_row.update(row[['lat', 'lon', 'z_egm96', 'dem_gnd_elev_egm96', 'lidar_obs_hgt', 'cluster']].to_dict())
    new_row['delta_Pos'] = np.nan
    new_row['delta_Hgt'] = np.nan
    new_row['delta_Elev'] = np.nan 
    new_row_df = pd.DataFrame([new_row])
    AIP_validated_df = pd.concat([AIP_validated_df, new_row_df], ignore_index=True)


In [53]:
AIP_validated_df

,geoLat,geoLong,txtName,Type of Obstacle,ValElev (m),valHgt (m),Location (inofficial),Description (inofficial),lat,lon,z_egm96,dem_gnd_elev_egm96,lidar_obs_hgt,cluster,delta_Pos,delta_Hgt,delta_Elev
0,50.941520,6.957296,NORDRHEIN-WESTFALEN 92-10,SPIRE,212.51,157.41,Köln,,50.941447,6.957300,211.935814,56.050893,155.884921,12.0,8.138802,-1.525079,-0.574186
1,51.000449,6.943347,NORDRHEIN-WESTFALEN 98-10,STACK,193.31,150.01,Koeln-Weidenpesch,,51.000461,6.943404,193.632675,42.401530,151.231145,0.0,4.233231,1.221145,0.322675
2,50.860811,6.980933,NORDRHEIN-WESTFALEN 163-20,INDUSTRIAL_SYSTEM,225.31,175.01,Godorf,KAMIN KONVERSIONSANLAGE,50.860858,6.980904,224.890246,55.266441,169.623805,99.0,5.574450,-5.386195,-0.419754
3,50.854561,6.980056,NORDRHEIN-WESTFALEN 163-30,INDUSTRIAL_SYSTEM,160.31,110.01,Godorf,KAMIN KRAFTWERK,50.854625,6.980029,163.384739,51.312915,112.071824,111.0,7.340018,2.061824,3.074739
4,50.857103,6.983283,NORDRHEIN-WESTFALEN 163-40,INDUSTRIAL_SYSTEM,160.31,110.01,Godorf,KAMIN XYLOL-ANLAGE,50.857168,6.983278,142.515626,49.932759,92.582867,107.0,7.314890,-17.427133,-17.794374
5,50.921800,6.932576,NORDRHEIN-WESTFALEN 167-10,BUILDING,184.71,133.01,Köln,,50.921806,6.932889,184.979678,65.143074,119.836604,37.0,21.969669,-13.173396,0.269678
6,50.902914,6.960628,NORDRHEIN-WESTFALEN 191-10,ANTENNA,190.76,138.01,Koeln-Raderberg,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,50.946944,6.931944,NORDRHEIN-WESTFALEN 204-10,TOWER,316.59,268.09,"Köln (Köln 8, Colonius), Gemark. Ehrenfeld, In...",,50.947134,6.931934,311.627979,52.220640,259.407339,9.0,21.071948,-8.682661,-4.962021
8,50.921617,7.004475,NORDRHEIN-WESTFALEN 263-10,TOWER,160.01,115.01,Koeln-Gremberg,,50.921623,7.004498,161.046235,46.006115,115.040120,38.0,1.781312,0.030120,1.036235
9,50.947908,6.942575,NORDRHEIN-WESTFALEN 271-10,ANTENNA,215.16,165.01,"Koeln, Fl.37, Flst.363,365",,50.948078,6.943082,215.396091,58.353737,157.042354,8.0,40.198364,-7.967646,0.236091


the differences in heights are problematic but could be due to the unknown DEM used for calculating the obstacles heights in the AIP. Or, the heights are surveyed. Need better DEM than EU-DEM 1.1

In [70]:
scattermapbox_objects = []

scattermapbox_objects.append(go.Scattermapbox(
    mode="markers",
    lon=lidar_validation_data['lon'], 
    lat=lidar_validation_data['lat'],
    marker={'size': 20, 'color': "red", 'opacity': 0.5,},
    text='Cluster: '+ lidar_validation_data['cluster'].apply(lambda x: str(x)) + '<br>' + 'Calculated height (m): ' + lidar_validation_data['lidar_obs_hgt'].apply(lambda x: str(round(x, 1))),
    hoverinfo='text'  # Only display the text on hover
))

scattermapbox_objects.append(go.Scattermapbox(
    mode="markers",
    lon=AIP_df['geoLong'], 
    lat=AIP_df['geoLat'],
    marker={'size': 10, 
            'symbol': 'circle',
            'color': "blue",
            'opacity': 0.7,
           },
    text='Name: ' + AIP_df['txtName']+'<br>'+'Published height (m): ' + AIP_df['valHgt (m)'].apply(lambda x: str(round(x, 1))),
    hoverinfo='text'  # Only display the text on hover
))

scattermapbox_objects.append(go.Scattermapbox(
        name = 'Data limits',
        mode="lines",
        line=dict(color="black", width=1),
        lat=np.array([LAT_MAX, LAT_MAX, LAT_MIN, LAT_MIN, LAT_MAX]),
        lon=np.array([LON_MIN, LON_MAX, LON_MAX, LON_MIN, LON_MIN]),
        hoverinfo='name',
        hoverlabel_namelength=-1   # https://stackoverflow.com/questions/36207887/plot-ly-hover-box-size-attribute
    ))

# Create a scatter plot of the highest points using Plotly with OpenStreetMap background
fig = go.Figure(data=scattermapbox_objects)

# Set the layout for the map
fig.update_layout(
    mapbox={
        'style': "open-street-map",
        'center': {'lon': np.mean(lidar_validation_data['lon']), 'lat': np.mean(lidar_validation_data['lat'])},
        'zoom': 12
    },
    showlegend=False
)

# Adjust the margins and set the height
fig.update_layout(height=800, margin={"r":10,"t":10,"l":10,"b":10})

# Show the figure
fig.show()